In [1]:
%matplotlib inline

In [1]:
# files and parameters

trajectory_file = 'Brownien_P1.txt'

cell_radii = [.03, .09, .27, .81]
location_counts = [20, 80, 320, 1280]

#

from tramway.helper import *
import os.path
import numpy as np

rwa_file = os.path.splitext(trajectory_file)[0]+'.rwa'
trajectory_file = os.path.join('input', trajectory_file)


In [3]:
# load the trajectories

if 'Brownien' in trajectory_file:

    # time step (please check)
    dt = 1.52587890625e-05
    # read the table
    xyt = load_xyt(trajectory_file, columns=['x', 'y', 'H'])
    # add missing columns
    xyt['n'] = np.ones(xyt.shape[0])
    xyt['t'] = np.arange(dt, (xyt.shape[0]+1)*dt, dt)

else:

    xyt = load_xyt(trajectory_file)


In [ ]:
# tessellate

analysis_tree = Analyses(xyt)

mesh_labels = []

for radius in cell_radii:
    cells = tessellate(xyt, 'hexagon', ref_distance=radius, min_location_count=0)
    analysis_tree.add(cells, label='hexagon_r_{:.2f}'.format(radius))

for count in location_counts:
    cells = tessellate(xyt, 'kmeans', ref_distance=0, avg_location_count=count,
            knn=(round(.75 * count), round(1.25 * count)), prune=False)
    analysis_tree.add(cells, label='kmeans_n_{:d}'.format(count))


In [ ]:
# save to file

save_rwa(rwa_file, analysis_tree)


In [ ]:
# or start from here and load the .rwa file

analysis_tree = load_rwa(rwa_file)


In [ ]:
# play

trajectories = analysis_tree.data

for mesh_label in analysis_tree.labels:
    
    cells = analysis_tree[mesh_label].data
    
    pass
